In [1]:
bucket = 'bucket-mldemo-dev-20200624145048345300000002'
 
# Define IAM role
import re


In [2]:
import boto3
boto3_session=boto3.Session(profile_name='ml-demo')

In [3]:
from sagemaker.session import Session
from sagemaker import get_execution_role

sagemaker_session=Session(boto3_session)

try:
    role = get_execution_role()
except:
    iam=boto3_session.client('iam')
    response=iam.get_role(
        RoleName='mldemo-dev-Sagemaker-Execution-Role'
    )
    role=response['Role']['Arn']
role

'arn:aws:iam::929759656386:role/mldemo-dev-Sagemaker-Execution-Role'

In [4]:
output_location = 's3://{}/output'.format(bucket)
output_location

's3://bucket-mldemo-dev-20200624145048345300000002/output'

In [5]:
from sagemaker.predictor import RealTimePredictor
linear_predictor=RealTimePredictor('mldemo-dev',sagemaker_session)

In [6]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [7]:
from s3fs.core import S3FileSystem
s3fs = S3FileSystem(anon=False,session=boto3_session)

import pandas as pd
df=pd.read_csv(s3fs.open('s3://%s/transformed/all_data.csv' % bucket),header=None)

df

,0,1,2,3,4,5,6,7,8,9,...,596,597,598,599,600,601,602,603,604,605
0,-18.0,18.0,335.39,386.34,-43.66,0.00,0.2575,-0.143333,-0.273333,0.270,...,2.780,2.790,2.20,2.305,2.39,2.310,2.890,2.440,2.400,0
1,-18.0,18.0,335.42,386.37,-43.63,0.03,0.2875,-0.113333,-0.243333,0.300,...,2.810,2.820,2.23,2.335,2.42,2.340,2.920,2.470,2.430,0
2,-18.0,18.0,335.31,386.26,-43.74,-0.08,0.1775,-0.223333,-0.353333,0.190,...,2.700,2.710,2.12,2.225,2.31,2.230,2.810,2.360,2.320,0
3,-18.0,18.0,334.46,385.41,-44.59,-0.93,-0.6725,-1.073333,-1.203333,-0.660,...,1.850,1.860,1.27,1.375,1.46,1.380,1.960,1.510,1.470,0
4,-18.0,18.0,335.34,386.29,-43.71,-0.05,0.2075,-0.193333,-0.323333,0.220,...,2.730,2.740,2.15,2.255,2.34,2.260,2.840,2.390,2.350,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,-18.0,18.0,352.29,403.24,-94.06,-2.23,-2.0800,-2.070000,-2.060000,-2.050,...,0.040,0.035,0.03,0.020,0.03,0.020,0.010,0.015,0.015,1
10101,-18.0,18.0,351.34,402.29,-95.01,-3.03,-3.0200,-3.010000,-3.000000,-2.985,...,-0.915,-0.920,-0.93,-0.920,-0.93,-0.940,-0.935,-0.935,-0.940,1
10102,-18.0,18.0,351.28,402.23,-95.07,-3.08,-3.0700,-3.060000,-3.045000,-3.050,...,-0.980,-0.990,-0.98,-0.990,-1.00,-0.995,-0.995,-1.000,-0.060,1
10103,-18.0,18.0,351.34,402.29,-95.01,-3.02,-3.0100,-3.000000,-2.985000,-2.990,...,-0.920,-0.930,-0.92,-0.930,-0.94,-0.935,-0.935,-0.940,0.000,1


In [9]:
test_data_df=df[round(len(df)*0.85+1):].copy().reset_index(drop=True)
test_data_df

,0,1,2,3,4,5,6,7,8,9,...,596,597,598,599,600,601,602,603,604,605
0,-18.0,18.0,329.88,380.83,-116.47,-0.370000,-0.515000,-0.516667,-0.520,-0.490,...,-0.400,-0.390,-0.29,-0.05,-0.31,0.020,-0.320,0.020,-0.160,1
1,-18.0,18.0,329.86,380.81,-116.49,-0.535000,-0.536667,-0.540000,-0.510,-0.570,...,-0.410,-0.310,-0.07,-0.33,0.00,-0.340,0.000,-0.180,-0.020,1
2,-18.0,18.0,329.89,380.84,-116.46,-0.506667,-0.510000,-0.480000,-0.540,-0.440,...,-0.280,-0.040,-0.30,0.03,-0.31,0.030,-0.150,0.010,0.030,1
3,-18.0,18.0,329.86,380.81,-116.49,-0.536667,-0.540000,-0.510000,-0.570,-0.470,...,-0.310,-0.070,-0.33,0.00,-0.34,0.000,-0.180,-0.020,0.000,1
4,-18.0,18.0,329.11,380.06,-117.24,-1.290000,-1.260000,-1.320000,-1.220,-1.400,...,-0.820,-1.080,-0.75,-1.09,-0.75,-0.930,-0.770,-0.750,-0.765,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,-18.0,18.0,352.29,403.24,-94.06,-2.230000,-2.080000,-2.070000,-2.060,-2.050,...,0.040,0.035,0.03,0.02,0.03,0.020,0.010,0.015,0.015,1
1511,-18.0,18.0,351.34,402.29,-95.01,-3.030000,-3.020000,-3.010000,-3.000,-2.985,...,-0.915,-0.920,-0.93,-0.92,-0.93,-0.940,-0.935,-0.935,-0.940,1
1512,-18.0,18.0,351.28,402.23,-95.07,-3.080000,-3.070000,-3.060000,-3.045,-3.050,...,-0.980,-0.990,-0.98,-0.99,-1.00,-0.995,-0.995,-1.000,-0.060,1
1513,-18.0,18.0,351.34,402.29,-95.01,-3.020000,-3.010000,-3.000000,-2.985,-2.990,...,-0.920,-0.930,-0.92,-0.93,-0.94,-0.935,-0.935,-0.940,0.000,1


In [10]:
data=','.join([str(x) for x in test_data_df.loc[0].tolist()[0:605]])
linear_predictor.predict(data)

{'predictions': [{'score': 0.004702974576503038, 'predicted_label': 0}]}

In [12]:
def eval_func(row):
    csv=','.join(row.tolist()[0:605])
    result=linear_predictor.predict(csv)
    return result
results=test_data_df.astype(str).apply(eval_func,axis=1)

In [13]:
predicted_labels=[1 if i['predictions'][0]['predicted_label']==1.0 else 0 for i in results]

In [14]:
score=[i['predictions'][0]['score'] for i in results]

In [15]:
test_data_df['predicted_labels']=predicted_labels
test_data_df['score']=score
test_data_df

,0,1,2,3,4,5,6,7,8,9,...,598,599,600,601,602,603,604,605,predicted_labels,score
0,-18.0,18.0,329.88,380.83,-116.47,-0.370000,-0.515000,-0.516667,-0.520,-0.490,...,-0.29,-0.05,-0.31,0.020,-0.320,0.020,-0.160,1,0,0.004703
1,-18.0,18.0,329.86,380.81,-116.49,-0.535000,-0.536667,-0.540000,-0.510,-0.570,...,-0.07,-0.33,0.00,-0.340,0.000,-0.180,-0.020,1,0,0.004644
2,-18.0,18.0,329.89,380.84,-116.46,-0.506667,-0.510000,-0.480000,-0.540,-0.440,...,-0.30,0.03,-0.31,0.030,-0.150,0.010,0.030,1,0,0.004993
3,-18.0,18.0,329.86,380.81,-116.49,-0.536667,-0.540000,-0.510000,-0.570,-0.470,...,-0.33,0.00,-0.34,0.000,-0.180,-0.020,0.000,1,0,0.004923
4,-18.0,18.0,329.11,380.06,-117.24,-1.290000,-1.260000,-1.320000,-1.220,-1.400,...,-0.75,-1.09,-0.75,-0.930,-0.770,-0.750,-0.765,1,0,0.003406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,-18.0,18.0,352.29,403.24,-94.06,-2.230000,-2.080000,-2.070000,-2.060,-2.050,...,0.03,0.02,0.03,0.020,0.010,0.015,0.015,1,1,0.006203
1511,-18.0,18.0,351.34,402.29,-95.01,-3.030000,-3.020000,-3.010000,-3.000,-2.985,...,-0.93,-0.92,-0.93,-0.940,-0.935,-0.935,-0.940,1,0,0.004011
1512,-18.0,18.0,351.28,402.23,-95.07,-3.080000,-3.070000,-3.060000,-3.045,-3.050,...,-0.98,-0.99,-1.00,-0.995,-0.995,-1.000,-0.060,1,0,0.004532
1513,-18.0,18.0,351.34,402.29,-95.01,-3.020000,-3.010000,-3.000000,-2.985,-2.990,...,-0.92,-0.93,-0.94,-0.935,-0.935,-0.940,0.000,1,0,0.004663


In [16]:
test_data_df[test_data_df['predicted_labels']==1]

,0,1,2,3,4,5,6,7,8,9,...,598,599,600,601,602,603,604,605,predicted_labels,score
23,-18.0,18.0,329.00,379.95,-117.35,-1.345000,-1.053333,-0.70,-0.69,-0.700,...,0.030,0.040,0.715,0.710,0.720,0.730,0.725,1,1,0.007476
24,-18.0,18.0,328.99,379.94,-117.36,-1.063333,-0.710000,-0.70,-0.71,-0.640,...,0.030,0.705,0.700,0.710,0.720,0.715,-0.010,1,1,0.006596
25,-18.0,18.0,328.98,379.93,-117.37,-0.720000,-0.710000,-0.72,-0.65,-0.930,...,0.695,0.690,0.700,0.710,0.705,-0.020,-0.010,1,1,0.006117
26,-18.0,18.0,328.97,379.92,-117.38,-0.720000,-0.730000,-0.66,-0.94,-0.795,...,0.680,0.690,0.700,0.695,-0.030,-0.020,-0.010,1,1,0.005907
27,-18.0,18.0,328.99,379.94,-117.36,-0.700000,-0.710000,-0.64,-0.92,-0.775,...,0.700,0.710,0.720,0.715,-0.010,0.000,0.010,1,1,0.005963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,-18.0,18.0,352.28,403.23,-94.07,-2.280000,-2.285000,-2.24,-2.09,-2.080,...,0.030,0.025,0.020,0.010,0.020,0.010,0.000,1,1,0.006033
1507,-18.0,18.0,352.27,403.22,-94.08,-2.295000,-2.250000,-2.10,-2.09,-2.080,...,0.015,0.010,0.000,0.010,0.000,-0.010,-0.005,1,1,0.006201
1508,-18.0,18.0,352.28,403.23,-94.07,-2.285000,-2.240000,-2.09,-2.08,-2.070,...,0.025,0.020,0.010,0.020,0.010,0.000,0.005,1,1,0.006231
1509,-18.0,18.0,352.27,403.22,-94.08,-2.250000,-2.100000,-2.09,-2.08,-2.070,...,0.010,0.000,0.010,0.000,-0.010,-0.005,-0.005,1,1,0.006144


In [27]:
most_certain=test_data_df.query('score>=0.0')
most_certain.sort_values('score',ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,598,599,600,601,602,603,604,605,predicted_labels,score
893,-18.0,18.0,357.00,407.95,-89.35,27.965,27.9250,28.0150,28.030,28.0400,...,3.290000,3.260000,3.250000,3.143333,2.910000,2.700000,2.220,1,1,0.496580
892,-18.0,18.0,356.97,407.92,-89.38,27.935,27.8950,27.9850,28.000,28.0100,...,3.260000,3.230000,3.220000,3.113333,2.880000,2.670000,2.190,1,1,0.493012
896,-18.0,18.0,357.81,408.76,-88.54,28.825,28.8400,28.8500,29.525,29.5200,...,4.060000,3.953333,3.720000,3.510000,3.030000,0.825000,0.790,1,1,0.407540
898,-18.0,18.0,357.74,408.69,-88.61,28.755,28.7700,28.7800,29.455,29.4500,...,3.990000,3.883333,3.650000,3.440000,2.960000,0.755000,0.720,1,1,0.399525
894,-18.0,18.0,357.02,407.97,-89.33,27.945,28.0350,28.0500,28.060,28.7350,...,3.280000,3.270000,3.163333,2.930000,2.720000,2.240000,0.035,1,1,0.392712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,-18.0,18.0,324.01,374.96,-122.34,-16.745,-16.5950,-16.4900,-16.480,-16.5825,...,-4.076667,-4.130000,-4.140000,-4.350000,-4.140000,-3.835000,-3.080,0,0,0.000322
445,-18.0,18.0,323.77,374.72,-122.58,-16.985,-16.8350,-16.7300,-16.720,-16.8225,...,-4.316667,-4.370000,-4.380000,-4.590000,-4.380000,-4.075000,-3.320,0,0,0.000288
450,-18.0,18.0,319.08,370.03,-127.27,-21.410,-21.5125,-21.3700,-21.320,-21.2900,...,-9.280000,-9.070000,-8.765000,-8.010000,-5.556667,-4.415000,-2.740,0,0,0.000213
451,-18.0,18.0,319.07,370.02,-127.28,-21.420,-21.5225,-21.3800,-21.330,-21.3000,...,-9.290000,-9.080000,-8.775000,-8.020000,-5.566667,-4.425000,-2.750,0,0,0.000212


In [28]:
# Importing the metrics package from sklearn library
from sklearn import metrics
# Creating the confusion matrix
cm = metrics.confusion_matrix(most_certain[605], most_certain['predicted_labels'])
# Assigning columns names
cm_df = pd.DataFrame(cm, 
            columns = ['Predicted Negative', 'Predicted Positive'],
            index = ['Actual Negative', 'Actual Positive'])
# Showing the confusion matrix
cm_df

,Predicted Negative,Predicted Positive
Actual Negative,241,201
Actual Positive,332,741


In [ ]:
most_certain.query('predicted_labels==1').describe()

In [ ]:
most_certain.query('predicted_labels==0').describe()